<a href="https://colab.research.google.com/github/VesalAhsani/Driver-behavior-detection/blob/main/DMS_MixedData_14classes_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check if the dataset zip file exists in Google Drive
import os
dataset_path = '/content/drive/MyDrive/Split.zip'  # Adjust the path if needed
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"The dataset file at {dataset_path} was not found!")
print("Dataset zip file exists.")

Dataset zip file exists.


In [3]:
# Unzipping the dataset
import zipfile
extract_path = '/content/SynDD2_mixed_640'
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Dataset unzipped successfully!")

Dataset unzipped successfully!


In [4]:
# Check the actual folder structure after unzipping
dataset_actual_path = os.path.join(extract_path, "Split")  # Adjust to the nested folder
if not os.path.exists(os.path.join(dataset_actual_path, "train")):
    raise FileNotFoundError("Train folder not found in the dataset path!")

In [5]:
# Install the YOLOv8 library
!pip install ultralytics --upgrade

# Import YOLO
from ultralytics import YOLO
import torch
import torch.nn as nn
import torch.nn.functional as F

print("Ultralytics library imported successfully!")

# Verify the dataset structure
from ultralytics.data.utils import check_cls_dataset
dataset_info = check_cls_dataset(dataset_actual_path)
print(f"Dataset verified: {dataset_info}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [6]:
# Count the number of training and validation images
import glob
train_images = glob.glob(os.path.join(dataset_actual_path, "train", "*", "*.jpg"))
val_images = glob.glob(os.path.join(dataset_actual_path, "val", "*", "*.jpg"))

if not train_images or not val_images:
    raise ValueError("Training or validation images were not found. Check dataset paths.")
print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")

Number of training images: 155465
Number of validation images: 17275


In [ ]:
# Define dataset path
DATASET_PATH = dataset_actual_path  # Path to the unzipped dataset
OUTPUT_PATH = '/content/drive/MyDrive/yolov8_cls_results_640'


# ✅ **Class Image Distribution (Training Counts)**
class_counts = {
    "Control_Panel": 11700,
    "Drinking": 11700,
    "Eating": 2689,  # ⚠️ Low sample class
    "Makeup": 11700,
    "Normal": 11700,
    "Phone_Call_(Left)": 11700,
    "Phone_Call_(Right)": 11700,
    "Reaching_Behind": 11700,
    "Sleep": 11700,
    "Smoking": 12376,
    "Talk_to_passengers": 11700,
    "Text_(Left)": 11700,
    "Text_(Right)": 11700,
    "Yawning": 11700,
}

# ✅ **Compute Alpha (Inverse Frequency Scaling)**
max_samples = max(class_counts.values())
alpha = torch.tensor([max_samples / class_counts[class_name] for class_name in class_counts], dtype=torch.float32).to("cuda")

print(f"✅ Calculated Alpha Weights: {alpha}")

# ✅ **Define Focal Loss**
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        """
        gamma: Focusing parameter (higher = more focus on hard examples)
        alpha: Class balancing factor (set to None or a tensor of class weights)
        reduction: 'mean' or 'sum'
        """
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        Compute Focal Loss.
        inputs: raw logits (before softmax)
        targets: true labels (as class indices)
        """
        log_probs = F.log_softmax(inputs, dim=-1)  # Convert logits to log probabilities
        probs = torch.exp(log_probs)  # Convert log probabilities to actual probabilities

        # Get the probability corresponding to the target class
        target_probs = probs.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)

        # Compute Focal Loss
        focal_factor = (1 - target_probs) ** self.gamma
        loss = -focal_factor * log_probs.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)

        if self.alpha is not None:
            alpha_factor = self.alpha.gather(dim=-1, index=targets.unsqueeze(-1)).squeeze(-1)
            loss *= alpha_factor

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

# ✅ **Initialize YOLOv8 Model**
model = YOLO('/content/drive/MyDrive/yolov8_cls_results_640/yolov8_cls_640_focal/weights/last.pt')  # Use yolov8m-cls as a starting point

# ✅ **Attach Focal Loss to YOLO**
model.loss = FocalLoss(gamma=2.0, alpha=alpha, reduction='mean')

# ✅ **Resume Training the Model**
model.train(
    data=DATASET_PATH,
    epochs=50,
    batch=32,
    imgsz=640,
    project=OUTPUT_PATH,
    name='yolov8_cls_640_focal',
    pretrained=False,
    optimizer='Adam',
    lr0=0.0005,
    resume=True,
)
print("✅ Training resumed!")

# ✅ **Evaluate the Model**
results = model.val()
print("✅ Validation completed!")

# ✅ **Save Best Weights**
best_weights_path = f"{OUTPUT_PATH}/yolov8_cls_640_focal/weights/best.pt"
os.makedirs(f"{OUTPUT_PATH}/yolov8_cls_640_focal/weights", exist_ok=True)
!cp /content/runs/classify/yolov8_cls_640_focal/weights/best.pt {best_weights_path}
print(f"✅ Best weights saved to {best_weights_path}")

✅ Calculated Alpha Weights: tensor([1.0578, 1.0578, 4.6025, 1.0578, 1.0578, 1.0578, 1.0578, 1.0578, 1.0578, 1.0000, 1.0578, 1.0578, 1.0578, 1.0578], device='cuda:0')
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=classify, mode=train, model=/content/drive/MyDrive/yolov8_cls_results_640/yolov8_cls_640_focal/weights/last.pt, data=/content/SynDD2_mixed_640/Split, epochs=50, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8_cls_results_640, name=yolov8_cls_640_focal, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/content/drive/MyDrive/yolov8_cls_results_640/yolov8_cls_640_focal/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=T

100%|██████████| 5.35M/5.35M [00:00<00:00, 32.2MB/s]


AMP: checks passed ✅


train: Scanning /content/SynDD2_mixed_640/Split/train... 155465 images, 0 corrupt: 100%|██████████| 155465/155465 [01:17<00:00, 2001.76it/s]


train: New cache created: /content/SynDD2_mixed_640/Split/train.cache


...


      Epoch    GPU_mem       loss  Instances       Size


      49/50      9.69G    0.08181          9        640: 100%|██████████| 4859/4859 [15:28<00:00,  5.23it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 270/270 [00:53<00:00,  5.07it/s]

                   all      0.996          1



      Epoch    GPU_mem       loss  Instances       Size


      50/50      10.1G    0.07619         32        640:  93%|█████████▎| 4506/4859 [14:18<00:54,  6.51it/s]